# Spark GraphX

In this notebook you will learn about GraphX, Spark's library for handling graphs. You will first use the basic functions on the simple graph modeling the Simpson family. Then you will use some graph algorithms on a larger graph loaded from files.

Let's start by importing the necessary classes. Execute the following cell.

In [2]:
import org.apache.spark.graphx._

## The Simpsons!

You will build the following graph representing a part of the Simpson family:
![graph](graph.png)

Vertices are marked with yellow circles and have IDs associated with them. Let's create this graph using this class for representing vertices:

In [3]:
case class Person(name:String, age:Int)

Create a `vertices` RDD by parallelizing an array of tuples, each containing the vertex ID (as a Long) and the matching Person object (use the figure as reference).

In [4]:
val vertices = sc.parallelize(Array((1L, Person("Homer", 39)),
    (2L, Person("Marge", 39)), (3L, Person("Bart", 12)),
    (4L, Person("Milhouse", 12))))

Next create an `edges` RDD containing `Edge` (which is a Spark GraphX class) elements. Each `Edge` object should be constructed by providing source vertex ID, destination vertex ID and the associated object (a String containing the relation name).

In [5]:
val edges = sc.parallelize(Array(Edge(4L, 3L, "friend"),
    Edge(3L, 1L, "father"), Edge(3L, 2L, "mother"),
    Edge(1L, 2L, "marriedTo")))

Finally, use Spark's `Graph` class to construct a graph from those two RDDs.

In [6]:
val graph = Graph(vertices, edges)

Use graph's `vertices` and `edges` methods to find out how many vertices and edges the graph has.

In [7]:
println(graph.vertices.count())
println(graph.edges.count())

4
4


## Mapping vertices and edges

Let's change String relation descriptions with these objects:

In [8]:
case class Relationship(relation:String)

Use `mapEdges` to create a new graph which has `Relationship` objects attached to its edges instead of Strings.

In [9]:
val newgraph = graph.mapEdges((partId, iter) => iter.map(edge => Relationship(edge.attr)))

Get the edges from the new graph (`collect`) to check if everything is ok.

In [10]:
newgraph.edges.collect()

Array(Edge(4,3,Relationship(friend)), Edge(3,1,Relationship(father)), Edge(3,2,Relationship(mother)), Edge(1,2,Relationship(marriedTo)))

We will now use the following class to extend the information about graph's vertices.

In [11]:
case class PersonExt(name:String, age:Int, children:Int=0, friends:Int=0, married:Boolean=false)

Create a new graph by mapping `Person` objects to `PersonExt` objects. Transfer the `name` and `age` fields and leave the rest at defaults.

In [12]:
val newGraphExt = newgraph.mapVertices((vid, person) => PersonExt(person.name, person.age))

## Aggregating messages

Use `aggregateMessages` method to produce new vertices with additional properties in `PersonExt` objects (number of children, number of friends and the married flag). Here is `aggregateMessages` method to refresh your memory:

```def aggregateMessages[A: ClassTag](
        sendMsg: EdgeContext[VD, ED, A] => Unit,
        mergeMsg: (A, A) => A,
        tripletFields: TripletFields = TripletFields.All)
    : VertexRDD[A]```

You need to provide `sendMsg` and `mergeMsg` functions. Model the `EdgeContext` to handle messages (and resulting vertex objects) of tuples each containing number of children, number of friends and the married flag (Int, Int, Boolean). This corresponds to the type `A` in the above definition. You should be able to provide `VD` and `ED` types. 
`sendMsg` function needs to check incoming context's attribute relation and depending on the type of relation call `sendToSrc` and/or `sendToDst` with the appropriate values.
`mergeMsg` function just needs to sum up all the incoming messages for a single vertex.

In [13]:
val aggVertices = newGraphExt.aggregateMessages(
    (ctx:EdgeContext[PersonExt, Relationship, Tuple3[Int, Int, Boolean]]) => {
        if(ctx.attr.relation == "marriedTo")
            { ctx.sendToSrc((0, 0, true)); ctx.sendToDst((0, 0, true)); }
        else if(ctx.attr.relation == "mother" || ctx.attr.relation == "father")
            { ctx.sendToDst((1, 0, false)); }
        else if(ctx.attr.relation == "friend")
            { ctx.sendToDst((0, 1, false)); ctx.sendToSrc((0, 1, false)); }
    },
    (msg1:Tuple3[Int, Int, Boolean], msg2:Tuple3[Int, Int, Boolean]) => (msg1._1+msg2._1, msg1._2+msg2._2, msg1._3 || msg2._3))

Print out the resulting vertices to check if you have the expected result.

In [14]:
aggVertices.collect.foreach(println)

(4,(0,1,false))
(1,(1,0,true))
(2,(1,0,true))
(3,(0,1,false))


## Joining graph data

Join this new `VertexRDD` containing calculated values with the graph it was created from using `outerJoinVertices`, shown here as a reminder:

```def outerJoinVertices[U:ClassTag, VD2:ClassTag](other: RDD[(VertexId, U)])
    (mapFunc: (VertexId, VD, Option[U]) => VD2) : Graph[VD2, ED]```

The resulting graph should have `PersonExt` objects attached to its vertices.

In [15]:
val graphAggr = newGraphExt.outerJoinVertices(aggVertices)(
    (vid, origPerson, optMsg) => { 
        optMsg match {
            case Some(msg) => PersonExt(origPerson.name, origPerson.age, msg._1, msg._2, msg._3)
            case _ => origPerson
    }}
)

Print all the resulting vertices to check the results.

In [16]:
graphAggr.vertices.collect().foreach(println)

(4,PersonExt(Milhouse,12,0,1,false))
(1,PersonExt(Homer,39,1,0,true))
(2,PersonExt(Marge,39,1,0,true))
(3,PersonExt(Bart,12,0,1,false))


## Graph algorithms

In this section you will use the “Human Wayfinding in Information Networks” dataset from Stanford University, which is a game that challenges a user to connect two Wikipedia articles with as few links as possible. Two files are important: `articles.tsv` and `links.tsv` ("tsv" stands for "tab-separated values"). `articles.tsv` contains just a list of all unique article names and `links.tsv` contains a two-by-two list of tab-separated names of articles that are linked.

Use the following snippet to load articles, remove empty lines and comments, and assign a unique ID to each article (change the path to where the `first-edition` GitHub repository is cloned if necessary):

In [17]:
val articles = sc.textFile("../first-edition/ch09/articles.tsv").
    filter(line => line.trim() != "" && !line.startsWith("#")).
    zipWithIndex().cache()

Load the links with this similar snippet:

In [18]:
val links = sc.textFile("../first-edition/ch09/links.tsv").
    filter(line => line.trim() != "" && !line.startsWith("#"))

Now an excercise in basic RDD operations. Parse each link line to obtain tuples of article names and then join that RDD with `articles` two times to replace article names with their IDs.

In [19]:
val linkIndexes = links.map(x => {
        val spl = x.split("\t");
        (spl(0), spl(1)) 
}).join(articles).map(x => x._2).
join(articles).map(x => x._2)

Now use the resulting RDD, containing source and destination article IDs, to construct a `Graph` using its `fromEdgeTuples` method.

In [20]:
val wikigraph = Graph.fromEdgeTuples(linkIndexes, 0)

## Shortest paths

Use `ShortestPaths` object to find the shortest path between two articles called "14th_century" and "Rainbow". 

First use the `articles` RDD to find IDs of those articles.

In [21]:
articles.filter(x => x._1 == "Rainbow" || x._1 == "14th_century").
    collect().foreach(println)

(14th_century,10)
(Rainbow,3425)


Run the shortest paths algorithm using one of those article IDs as a starting point (first import `ShortestPaths` from `org.apache.spark.graphx.lib`). Search the results to find the value corresponding to the second article. You might need to consult the [documentation](https://spark.apache.org/docs/2.2.0/api/scala/index.html#org.apache.spark.graphx.lib.ShortestPaths$).

What is the shortest path between those two articles?

In [22]:
import org.apache.spark.graphx.lib._
val shortest = ShortestPaths.run(wikigraph, Seq(10))
shortest.vertices.filter(x => x._1 == 3425).collect.foreach(println)

(3425,Map(10 -> 2))


## Page rank

Run the [Page rank algorithm](https://spark.apache.org/docs/2.0.0/api/scala/index.html#org.apache.spark.graphx.GraphOps@pageRank) on the graph with tolerance of `0.001`.

In [23]:
val ranked = wikigraph.pageRank(0.001)

Find the top five ranked pages by querying the vertices RDD of the resulting graph. (You will have to implement a custom Scala `Ordering`.)

In [24]:
val ordering = new Ordering[Tuple2[VertexId,Double]] {
    def compare(x:Tuple2[VertexId, Double], y:Tuple2[VertexId, Double]):Int =
    x._2.compareTo(y._2) }
val top10 = ranked.vertices.top(10)(ordering)

Join these results with the articles RDD to find the matching article names.

In [25]:
sc.parallelize(top10).join(articles.map(_.swap)).collect.
sortWith((x, y) => x._2._1 > y._2._1).foreach(println)

(4297,(43.79545390514789,United_States))
(1568,(29.5193874977993,France))
(1433,(29.090727549623136,Europe))
(4293,(28.602299291981907,United_Kingdom))
(1389,(22.334694754966193,English_language))
(1694,(22.14622664140612,Germany))
(4542,(21.690337661883135,World_War_II))
(1385,(20.480194747598016,England))
(2417,(20.226473566280973,Latin))
(2098,(18.55611493055244,India))


## Connected components

Finding connected components of a graph means finding subgraphs in which every vertex can be reached from every other vertex by following the edges in the subgraph. A graph is *connected* if it contains only one connected component and all of its vertices can be reached from every other vertex. 

Find connected components in the Wikispeedia graph by calling its `connectedComponents` method and save the result in the variable `wikiCC`.

In [26]:
val wikiCC = wikigraph.connectedComponents()

Property objects of the `wikiCC` graph’s vertices contain the lowest vertex ID of the connected component to which they belong. To find the connected components, find distinct values of vertices in the `wikiCC` graph and join them with `articles` RDD.

In [27]:
wikiCC.vertices.map(x => (x._2, x._2)).
    distinct().join(articles.map(_.swap)).collect.foreach(println)

(0,(0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in))
(1210,(1210,Directdebit))


How many vertices are there in each connected component (count vertices by the attached property objects).

In [28]:
wikiCC.vertices.map(x => (x._2, x._2)).countByKey().foreach(println)

(0,4589)
(1210,3)


## Strongly connected components

Strongly connected components (SCCs) are subgraphs where all vertices are connected to every other vertex in the subgraph (not necessarily directly). All vertices in an SCC need to be reachable from each other (following the direction of the edges).

Find strongly connected components of the Wikispeedia graph by calling the `stronglyConnectedComponents` method and specifying 100 as the maximum number of iterations.

In [29]:
val wikiSCC = wikigraph.stronglyConnectedComponents(100)

Find the number of strongly connected components as you did for connected components.

In [30]:
wikiSCC.vertices.map(x => x._2).distinct.count

519

Find the strongly connected components with the largest number of vertices (sort them by the number of vertices).

In [31]:
wikiSCC.vertices.map(x => (x._2, x._1)).countByKey().
filter(_._2 > 1).toList.sortWith((x, y) => x._2 > y._2).foreach(println)

(6,4051)
(2488,6)
(1831,3)
(892,2)
(1950,2)
(4224,2)
(1111,2)
(2474,2)
(477,2)
(557,2)
(2160,2)
(1834,2)
(2251,2)
(1513,2)
(2142,2)
(1986,2)
(195,2)
(1976,2)
(2321,2)


What are the names of those articles?

In [32]:
wikiSCC.vertices.filter(x => x._2 == 2488).
    join(articles.map(x => (x._2, x._1))).collect.foreach(println)

(2488,(2488,List_of_African_countries))
(2496,(2488,List_of_Oceanian_countries))
(2493,(2488,List_of_European_countries))
(2498,(2488,List_of_South_American_countries))
(2490,(2488,List_of_Asian_countries))
(2495,(2488,List_of_North_American_countries))
